In [1]:
from elasticsearch import Elasticsearch, helpers
from stop_words import get_stop_words
import spacy
import time
import json
import re

In [5]:
list_ents=set(parser.entity.cfg["actions"]["1"])

In [2]:
es=Elasticsearch([ {'host': host, 'port': n_port, 'url_prefix': 'es'} ])

In [3]:
stop_words = get_stop_words('en')

In [4]:
parser = spacy.load('en')

In [6]:
N=es.count(index=name_index, doc_type="email")["count"]

In [ ]:
doc=es.get(index=name_index, doc_type='email', id="1")

In [ ]:
text=doc["_source"]

In [ ]:
content=text["contents"].strip()

In [ ]:
def show(doc):
    kk=list(doc["_source"].keys())
    for k in kk:
        print(k + ": " + str(doc["_source"][k]))
    return None

In [ ]:
doc_processed=re.sub("\[IMAGE\]|\|", "", doc_processed)

In [8]:
def process_document(text):
    '''
    input: output of parser
    '''
    ents={}
    pos={}
    for ent in text.ents:
        if ent.label_ not in ents:
            ents[ent.label_] = []
        ents[ent.label_].append(ent.text.strip())
        
    for token in text:
        if token.pos_ in ["ADJ", "NOUN", "VERB"]:
            if token.pos_ not in pos:
                pos[token.pos_]=[]
            pos[token.pos_].append(token.text.strip())
            
    nlp = { "__nlp__": {"ents":{k.lower(): ents[k] for k in ents}, "pos":{k.lower(): pos[k] for k in pos}} }
    return nlp

In [9]:
def clean(content):
    content.strip()
    content=re.sub("\[IMAGE\]|\|", "", content)
    content=content.strip()
    return content

In [ ]:
start=time.time()

docs=[]
actions=[]
count=0
chunk_size_scan=1000
chunk_size_bulk=10
batch_size=100
remove=["9", "", "!", "(lavorato's office)", "(maureen x31808)", "1 -\r\n 3pm", "3 - 3", "?"]
try:
    for doc in helpers.scan(es, index=name_index, doc_type='email', size=chunk_size_scan):
        count+=1
        if count % chunk_size_scan == 0:
            print(count)
        docs.append(doc)
        if len(docs) == chunk_size_scan:
            texts = [clean(doc["_source"]["contents"]) for doc in docs]
            i=0
            for doc_processed in parser.pipe(texts, batch_size=batch_size, n_threads=4):
                nlp=process_document(doc_processed)
                docs[i]["_source"].update(nlp)
                for k in remove:
                    if k in docs[i]["_source"]:
                        del docs[i]["_source"][k]
                v = {
                        '_op_type': 'create',
                        '_index': name_index_nlp,
                        '_type': 'email',
                        '_id': docs[i]["_id"],
                        '_source': docs[i]["_source"]
                }
                #print(docs[i]["_id"])
                #print(docs[i]["_source"])
                actions.append(v)
                i+=1
            print(docs[i-1]["_id"])
            print()
            for b in helpers.parallel_bulk(es, actions=actions, thread_count=4, chunk_size=chunk_size_bulk):
                pass
            docs=[]
            actions=[]
except KeyboardInterrupt:
    end=time.time()
    print((end-start))
    print("exit")

In [ ]:
def remove_not_ascii(string):
    string = string.encode('utf-8',errors='ignore').decode()
    return string